In [ ]:
using ADCME
using Plots
using PyCall
#### Read data
py"""
import numpy as np
"""

In [ ]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7]/T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(pr) / log(10) + c) / (n - 0.14 *(log(pr) / log(10) + c))
    return  10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2))
end

In [ ]:
V = 1  # Volume
m = 0.25053305505123996  # Total mass
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"("data/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
T = 1200.0 # Temperature
Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
#Y .+= 0.00001
#Y ./ sum(Y)
### Quantities that depend on the Unknowns
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
tbd = Int.(tbd)
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
falofr = Int.(falofr)
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
elmr = Int.(elmr)

order = py"np.load"("data/reaction_orders.npy")
#Af = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
#β = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
#E = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"("data/reaction_orders_t.npy")
efficiency_t = py"np.load"("data/efficiency_t.npy")
Af_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"("data/reaction_orders_f.npy")
troefall = py"np.load"("data/troefall.npy") .+ 1
troefall_coeff = py"np.load"("data/troefall_coeff.npy")
efficiency_f = py"np.load"("data/efficiency_f.npy")
Af_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_hi.npy") 

Af_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f;

In [ ]:
function f(TY, θ)
    T = TY[1]
    Y = TY[2:end]
    ρ = m / V # density
    X = tf.reshape(ρ * tf.divide(Y, W), (N,1)) # Concentration
    Q = ADCME.constant(ones(M)) # Individual progress rates
    islarge = T > ADCME.constant(NASA_coeffs[:,1])
    islarge = convert_to_tensor(islarge, dtype=Float64)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X[:,1]
    Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
    Qtbd =  Kf_t .* (prod(tf.pow(X, order_t), dims=1)) .- Kr_t .* (prod(tf.pow(X, ν2[:,tbd]), dims=1) .* reversible[tbd])
    M_f = efficiency_f' * X[:,1]
    Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
    Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ADCME.constant(ones(size(falofr)[1]))
    for (i,s) in enumerate(troefall)
        a = troefall_coeff[:,i]
        fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
        c = -0.4 - 0.67 * log(fcent) / log(10)
        n = 0.75 - 1.27 * log(fcent) / log(10)
        f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
        Fac = scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
    end
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
    Qfalofr = Kf_f .* (prod(tf.pow(X, order_f), dims=1)) .- Kr_f .* (prod(tf.pow(X, ν2[:,falofr]), dims=1) .* reversible[falofr])
    ####
    Af = exp(θ[:,1])
    β = zeros(M)
    E = exp(θ[:,3])
    Kf = Af .* (T ^ β) .* exp(-E / (R * T))
    Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
    Qelmr = Kf .* (prod(tf.pow(X, order), dims=1)) .- Kr .* (prod(tf.pow(X, ν2[:,elmr]), dims=1) .* reversible[elmr])
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h / W - R ./ W * T   # Internal energy for species
    p = sum(X) * R * T # pressure
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### Species Conservation
    mgen_dot = V * ω_dot
    Y_dot = (1 / m) * ((min_dot * (Yin - Y) - mout_dot * Y) + mgen_dot) 
    ###### EnergyConservation
    T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return tf.concat([tf.reshape(T_dot, (1,)), Y_dot], 0)
end

In [50]:
timestep = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/timestep.npy")
Tt_cant = py"np.load"("data/temperature.npy")
T_ref = zeros(2001)
T_ref[2:end] = Tt_cant
T_ref[1] = 1200
Yt_cant = py"np.load"("data/mass_fraction.npy")
Y_ref = [Y Yt_cant]
obs = [T_ref Y_ref']

2001×7 Array{Float64,2}:
 1200.0   0.166667    0.166667  0.166667  0.166667  0.166667  0.166667
 1200.18  0.166647    0.166671  0.166665  0.166645  0.166706  0.166667
 1200.36  0.166628    0.166674  0.166663  0.166623  0.166745  0.166667
 1200.54  0.166609    0.166678  0.166662  0.166601  0.166784  0.166667
 1200.72  0.166589    0.166682  0.16666   0.166579  0.166823  0.166667
 1200.9   0.16657     0.166686  0.166658  0.166557  0.166862  0.166667
 1201.08  0.16655     0.16669   0.166656  0.166535  0.166901  0.166667
 1201.26  0.166531    0.166694  0.166655  0.166514  0.16694   0.166667
 1201.44  0.166512    0.166698  0.166653  0.166492  0.16698   0.166667
 1201.62  0.166492    0.166702  0.166651  0.16647   0.167019  0.166667
 1201.8   0.166473    0.166706  0.166649  0.166448  0.167058  0.166667
 1201.98  0.166453    0.16671   0.166648  0.166426  0.167097  0.166667
 1202.16  0.166433    0.166713  0.166646  0.166404  0.167137  0.166667
    ⋮                                               

In [56]:
Afr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
βr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
Er = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ
pr = [log.(Afr) βr log.(Er)]
pr[1,1] = 28.25
pr[2,1] = 17.75
pr[1,3] = 18.75
pr[2,3] = 17.75

17.75

In [57]:
nt = size(timestep)[1]
TY = [T; Y]
function condition(k, U, t, θ)
    k <= nt - 1
end

function body(k, U, t, θ)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    k1 = f(uk, θ)
    k2 = f(uk + 0.5 * dt * k1, θ)
    k3 = f(uk + 0.5 * dt * k2, θ)
    k4 = f(uk + dt * k3, θ) 
    u_new = uk + 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    U = write(U, k+1, u_new)
    k+1, U, t, θ
end
k = constant(1, dtype=Int32)
t = constant(timestep)
θ = Variable(pr)
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t, θ])
U_array = set_shape(stack(U_out), (nt, N+1))
loss = sum((U_array[inds,:]-obs[inds,:])^2) 
sess = Session()
init(sess)
#u_out = run(sess, U_array)
loss = BFGS!(sess, loss)
out = run(sess, θ)

iter 0, current loss=362397.34121039574
iter 1, current loss=8.54203387794032e6
iter 2, current loss=6.543404235195075e6
iter 3, current loss=115843.23082808303
================ STEP 0 ===============
iter 4, current loss=10451.735771084492
================ STEP 1 ===============
iter 5, current loss=6117.71802931791
================ STEP 2 ===============
iter 6, current loss=6103.412267572623
================ STEP 3 ===============
iter 7, current loss=6075.061536977392
================ STEP 4 ===============
iter 8, current loss=5989.851438792128
================ STEP 5 ===============
iter 9, current loss=5772.825829301961
================ STEP 6 ===============
iter 10, current loss=5109.523892490345
================ STEP 7 ===============
iter 11, current loss=1162.900525046948
iter 12, current loss=233692.46942704136
iter 13, current loss=475.112930562076
================ STEP 8 ===============
iter 14, current loss=73221.97353324351
iter 15, current loss=198.65141945899265
====

┌ Info: Optimization starts...
└ @ ADCME /home/darve/adncat/.julia/packages/ADCME/G762I/src/optim.jl:284


2×3 Array{Float64,2}:
 28.3244  0.0  18.8021
 17.9617  0.0  17.7315

In [58]:
nt = size(timestep)[1]
function condition(k, U, t, θ)
    k <= nt - 1
end

function body(k, U, t, θ)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    k1 = f(uk, θ)
    k2 = f(uk + 0.5 * dt * k1, θ)
    k3 = f(uk + 0.5 * dt * k2, θ)
    k4 = f(uk + dt * k3, θ) 
    u_new = uk + 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    U = write(U, k+1, u_new)
    k+1, U, t, θ
end
k = constant(1, dtype=Int32)
t = constant(timestep)
θ = Variable(out)
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t, θ])
U_array = set_shape(stack(U_out), (nt, N+1))
sess = Session()
init(sess)
u_out = run(sess, U_array)

2001×7 Array{Float64,2}:
 1200.0   0.166667    0.166667  0.166667  0.166667  0.166667  0.166667
 1200.18  0.166647    0.166671  0.166665  0.166645  0.166706  0.166667
 1200.36  0.166628    0.166674  0.166663  0.166623  0.166745  0.166667
 1200.54  0.166609    0.166678  0.166662  0.166601  0.166784  0.166667
 1200.72  0.166589    0.166682  0.16666   0.166579  0.166823  0.166667
 1200.9   0.16657     0.166686  0.166658  0.166557  0.166862  0.166667
 1201.08  0.16655     0.16669   0.166656  0.166535  0.166901  0.166667
 1201.26  0.166531    0.166694  0.166655  0.166514  0.16694   0.166667
 1201.44  0.166511    0.166698  0.166653  0.166492  0.16698   0.166667
 1201.62  0.166492    0.166702  0.166651  0.16647   0.167019  0.166667
 1201.8   0.166473    0.166706  0.166649  0.166448  0.167058  0.166667
 1201.98  0.166453    0.16671   0.166648  0.166426  0.167097  0.166667
 1202.16  0.166433    0.166713  0.166646  0.166404  0.167137  0.166667
    ⋮                                               

In [ ]:
#dt = 1e-6
nt = size(timestep[1:1000])[1]
#### Implicit scheme, need to refine for variable time step and order
function res_and_jac(param, x)
    dt = param[1]
    x0 = param[2:end]
    res = x - x0 - dt * f(x)
    jac = gradients(res, x)
    res, jac 
end

function condition(k, U, t)
    k <= nt - 1
end

function body(k, U, t)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    ADCME.options.newton_raphson.rtol = 1e-5 # relative tolerance
    ADCME.options.newton_raphson.tol = 1e-5 # absolute tolerance
    ADCME.options.newton_raphson.verbose = true # print details in newton_raphson
    param = tf.concat([tf.reshape(dt, (1,)), uk], 0)
    u_new = newton_raphson_with_grad(res_and_jac, constant(uk), param)
    U = write(U, k+1, u_new)
    k+1, U, t
end

k = constant(1, dtype=Int32)
t = constant(timestep)
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t])
U_array = set_shape(stack(U_out), (nt, N+1))
sess = Session()
init(sess)
u_out = run(sess, U_array)

In [70]:
inds = range(1,2000, step=100)
plot(timestep[inds], Y_ref'[inds,:], seriestype = :scatter, legend = false)
plot!(timestep, u_out[:,2:end])
xlabel!("Time")
ylabel!("Mass Fractions")
savefig("Mass_fraction.pdf")

In [71]:
inds = range(1,2000, step=100)
plot(timestep[inds], T_ref[inds], seriestype = :scatter, label = "observation")
plot!(timestep, u_out[:,1], label = "prediction")
xlabel!("Time")
ylabel!("Temperature")
savefig("temperature.pdf")

In [ ]:
dt = 1e-6
u1 = constant(TY)
function res_and_jac(x0, x)
    res = x - x0 - dt * f(x)
    jac = gradients(res, x)
    res, jac 
end
ADCME.options.newton_raphson.rtol = 1e-4 # relative tolerance
ADCME.options.newton_raphson.tol = 1e-4 # absolute tolerance
ADCME.options.newton_raphson.verbose = true # print details in newton_raphson
u_est = newton_raphson_with_grad(res_and_jac, constant(u1), u1)

In [ ]:
sess = Session()
init(sess)
u_e = run(sess, u_est)

In [ ]:
28.3244  0.0  18.8021
 17.9616  0.0  17.7315

1:100:1901

In [17]:
u_out[range(1,2000, step=100),2:end]

20×6 Array{Float64,2}:
 0.166667     0.166667  0.166667  0.166667  0.166667  0.166667
 0.164636     0.167095  0.166476  0.164443  0.170684  0.166667
 0.16238      0.167626  0.16624   0.162145  0.174944  0.166667
 0.159845     0.168292  0.165943  0.159778  0.179476  0.166667
 0.156957     0.169139  0.165566  0.157356  0.184315  0.166667
 0.153612     0.170233  0.165079  0.154901  0.189508  0.166667
 0.149653     0.171674  0.164437  0.152453  0.195116  0.166667
 0.144837     0.173623  0.163569  0.150081  0.201223  0.166667
 0.138759     0.17635   0.162355  0.147919  0.20795   0.166667
 0.130703     0.18034   0.160578  0.146224  0.215488  0.166667
 0.119258     0.186569  0.157805  0.145556  0.224146  0.166667
 0.101352     0.197192  0.153075  0.147239  0.234476  0.166667
 0.0703485    0.216943  0.144281  0.154382  0.247379  0.166667
 0.02572      0.246358  0.131184  0.167719  0.262352  0.166667
 0.0036186    0.260282  0.124984  0.172326  0.272122  0.166667
 0.000323827  0.261988  0.124225

In [38]:
pr = [log.(Afr) βr log.(Er)]

2×3 Array{Float64,2}:
 28.3242  0.0  18.8021
 17.9625  0.0  17.7317

In [61]:
loss

61-element Array{Any,1}:
 115843.23082808303
  10451.735771084492
   6117.71802931791
   6103.412267572623
   6075.061536977392
   5989.851438792128
   5772.825829301961
   5109.523892490345
    475.112930562076
    198.65141945899265
    160.45410954586762
    154.95734368943317
    150.72196383322716
      ⋮
      0.09133938104300723
      0.09016275183089376
      0.0883161836260606
      0.07889503924849452
      0.059865418822947665
      0.04074770793237236
      0.016361935821165473
      0.003948122152681382
      8.758926619680821e-5
      8.756977793124255e-5
      8.756890338121464e-5
      8.756890337962756e-5

In [73]:
plot(log.(loss), label="log of mse")
xlabel!("steps")
ylabel!("logarithm of loss")
savefig("loss.pdf")